In [2]:
# Preliminaries -----------------------------------------------------------
if (!require("pacman")) install.packages("pacman")
pacman::p_load(tidyverse, ggplot2, dplyr, lubridate, stringr, readxl, data.table, gdata, purrr)
source("functions.R")

Loading required package: pacman



In [24]:
# 1. Define years and file names
target_years <- 2014:2019
file_list <- paste0("../data/output/data-", target_years, ".csv")

# 2. Merge logic
final_ma <- file_list %>%
  set_names(target_years) %>%
  map_dfr(~ {
    if (file.exists(.x)) {
      read_csv(.x, col_types = cols(.default = "c")) 
    } else {
      stop(paste("Missing file:", .x))
    }
  }, .id = "source_year") %>%
  # 1. Convert columns to numeric
  mutate(across(c(avg_enrollment, premium, bid, avg_ffscost, basic_premium), as.numeric)) %>%
  # 2. Drop the redundant .y columns first
  select(-ends_with(".y")) %>%
  # 3. Drop the 'clean' year column so it doesn't conflict with year.x
  select(-any_of("year")) %>%
  # 4. Now rename year.x to year (and any others like state.x to state)
  rename_with(~str_remove(., "\\.x$"), ends_with(".x"))

In [26]:
# 5. Save the clean version
write_excel_csv(final_ma, "../data/output/final_ma_data.csv")

In [25]:
summary(final_ma %>% select(premium, bid, avg_enrollment))

    premium            bid         avg_enrollment    
 Min.   :  0.00   Min.   : 190.6   Min.   :   11.00  
 1st Qu.:  0.00   1st Qu.: 705.6   1st Qu.:   29.67  
 Median : 44.00   Median : 773.3   Median :   85.67  
 Mean   : 59.43   Mean   : 779.0   Mean   :  463.21  
 3rd Qu.: 89.00   3rd Qu.: 846.6   3rd Qu.:  288.08  
 Max.   :388.00   Max.   :1881.8   Max.   :82187.33  
 NA's   :241513   NA's   :52518    NA's   :284250    